In [ ]:
import os
import pandas as pd
import yfinance as yf
import jquantsapi
from dotenv import load_dotenv
import numpy as np
from datetime import datetime, timedelta
import warnings
import traceback
warnings.filterwarnings('ignore')

class StockAnalyzer:
    def __init__(self, stock_code):
        self.stock_code = stock_code
        self.ticker_symbol = f"{stock_code}.T"
        
        # jQuants API接続
        load_dotenv()
        my_mail_address = os.environ['JQUANTS_EMAIL']
        my_password = os.environ['JQUANTS_PASSWORD']
        self.cli = jquantsapi.Client(mail_address=my_mail_address, password=my_password)
        
        # yfinance接続
        self.ticker = yf.Ticker(self.ticker_symbol)
        
        self.analysis_results = {}
    
    def get_basic_info(self):
        """基本情報取得"""
        print("=== 基本情報取得中 ===")
        
        # jQuantsから基本情報
        try:
            company_info = self.cli.get_listed_info(code=self.stock_code)
            self.analysis_results['company_info'] = company_info
        except Exception as e:
            print(f"jQuants基本情報取得エラー: {e}")
        
        # yfinanceから基本情報
        try:
            info = self.ticker.info
            self.analysis_results['yf_info'] = info
            print(f"企業名: {info.get('longName', 'N/A')}")
            print(f"業種: {info.get('sector', 'N/A')}")
            print(f"時価総額: {info.get('marketCap', 'N/A'):,} 円" if info.get('marketCap') else "時価総額: N/A")
        except Exception as e:
            print(f"yfinance基本情報取得エラー: {e}")
    
    def get_financial_data(self):
        """財務データ取得・分析"""
        print("\n=== 財務データ分析中 ===")
        
        # jQuantsから財務データ
        try:
            # 過去3年分の財務データを取得
            end_date = datetime.now()
            start_date = end_date - timedelta(days=1095)  # 3年前
            
            financials = self.cli.get_fins_statements(
                code=self.stock_code,
                # date_from=start_date.strftime('%Y-%m-%d'),
                # date_to=end_date.strftime('%Y-%m-%d')
            )
            
            # データフレームが空でないことを確認
            if not financials.empty:
                # 最新の財務データ（最初の行）
                latest_financial = financials.query('TypeOfDocument.str.contains("FinancialStatements_Consolidated")').iloc[-1]
                self.analysis_results['financial_data'] = financials
                
                # 主要財務指標の計算
                revenue = int(latest_financial.get('NetSales', 0))
                operating_income = int(latest_financial.get('OperatingProfit', 0))
                net_income = int(latest_financial.get('Profit', 0))
                total_assets = int(latest_financial.get('TotalAssets', 0))
                shareholders_equity = int(latest_financial.get('Equity', 0))
                
                print(f"売上高: {revenue:,} 円")
                print(f"営業利益: {operating_income:,} 円")
                print(f"当期純利益: {net_income:,} 円")
                print(f"総資産: {total_assets:,} 円")
                print(f"純資産: {shareholders_equity:,} 円")
                
                # 財務指標計算
                if revenue > 0:
                    operating_margin = (operating_income / revenue) * 100
                    print(f"営業利益率: {operating_margin:.2f}%")
                
                if total_assets > 0:
                    roa = (net_income / total_assets) * 100
                    print(f"ROA: {roa:.2f}%")
                
                if shareholders_equity > 0:
                    roe = (net_income / shareholders_equity) * 100
                    print(f"ROE: {roe:.2f}%")
                
            else:
                print("財務データが見つかりませんでした。")
                
        except Exception as e:
            print(f"財務データ取得エラー: {e}")

    
    def get_stock_price_analysis(self):
        """株価分析"""
        print("\n=== 株価分析中 ===")
        
        try:
            # 過去1年の株価データ取得
            hist = self.ticker.history(period="1y")
            self.analysis_results['price_history'] = hist
            
            if not hist.empty:
                current_price = hist['Close'].iloc[-1]
                highest_price = hist['High'].max()
                lowest_price = hist['Low'].min()
                avg_volume = hist['Volume'].mean()
                
                print(f"現在株価: {current_price:.2f} 円")
                print(f"52週高値: {highest_price:.2f} 円")
                print(f"52週安値: {lowest_price:.2f} 円")
                print(f"平均出来高: {avg_volume:,.0f} 株")
                
                # テクニカル指標
                hist['SMA20'] = hist['Close'].rolling(window=20).mean()
                hist['SMA50'] = hist['Close'].rolling(window=50).mean()
                
                sma20 = hist['SMA20'].iloc[-1]
                sma50 = hist['SMA50'].iloc[-1]
                
                print(f"20日移動平均: {sma20:.2f} 円")
                print(f"50日移動平均: {sma50:.2f} 円")
                
                # RSI計算
                delta = hist['Close'].diff()
                gain = (delta.where(delta > 0, 0)).rolling(window=14).mean()
                loss = (-delta.where(delta < 0, 0)).rolling(window=14).mean()
                rs = gain / loss
                rsi = 100 - (100 / (1 + rs))
                current_rsi = rsi.iloc[-1]
                
                print(f"RSI (14日): {current_rsi:.2f}")
                
        except Exception as e:
            print(f"株価分析エラー: {e}")
    
    def get_market_sentiment(self):
        """市場センチメント分析"""
        print("\n=== 市場センチメント分析中 ===")
        
        try:
            # アナリスト推奨情報
            recommendations = self.ticker.recommendations
            if recommendations is not None and not recommendations.empty:
                latest_rec = recommendations.iloc[-1]
                
                # アナリストの推奨分布を分析
                total_recommendations = latest_rec['strongBuy'] + latest_rec['buy'] + latest_rec['hold'] + latest_rec['sell'] + latest_rec['strongSell']
                
                # 買い推奨率の計算
                buy_ratio = (latest_rec['strongBuy'] * 2 + latest_rec['buy']) / total_recommendations * 100
                sell_ratio = (latest_rec['strongSell'] * 2 + latest_rec['sell']) / total_recommendations * 100
                
                print("アナリスト推奨分析:")
                print(f"  強い買い: {latest_rec['strongBuy']}件")
                print(f"  買い: {latest_rec['buy']}件")
                print(f"  ホールド: {latest_rec['hold']}件")
                print(f"  売り: {latest_rec['sell']}件")
                print(f"  強い売り: {latest_rec['strongSell']}件")
                
                print(f"\n買い推奨率: {buy_ratio:.1f}%")
                print(f"売り推奨率: {sell_ratio:.1f}%")
                
                # 推奨度の判定
                if buy_ratio > 70:
                    recommendation = "非常に強い買い推奨"
                elif buy_ratio > 50:
                    recommendation = "買い推奨"
                elif buy_ratio > 30:
                    recommendation = "やや買い推奨"
                elif sell_ratio > 70:
                    recommendation = "非常に強い売り推奨"
                elif sell_ratio > 50:
                    recommendation = "売り推奨"
                elif sell_ratio > 30:
                    recommendation = "やや売り推奨"
                else:
                    recommendation = "中立"
                
                print(f"総合アナリスト推奨: {recommendation}")
                
                self.analysis_results['recommendations'] = {
                    'details': latest_rec,
                    'buy_ratio': buy_ratio,
                    'sell_ratio': sell_ratio,
                    'recommendation': recommendation
                }
            
            # 機関投資家保有情報
            institutional_holders = self.ticker.institutional_holders
            if institutional_holders is not None:
                print(f"\n機関投資家数: {len(institutional_holders)}")
                self.analysis_results['institutional_holders'] = institutional_holders
                
        except Exception as e:
            print(f"市場センチメント分析エラー: {e}")

    
    def calculate_investment_score(self):
        """投資スコア計算"""
        print("\n=== 総合投資判定 ===")
        
        scores = []
        reasons = []
        
        try:
            # 1. 財務健全性スコア (0-25点)
            financial_score = 0
            if 'financial_data' in self.analysis_results and len(self.analysis_results['financial_data']):
                latest_financial = self.analysis_results['financial_data'].query('TypeOfDocument.str.contains("FinancialStatements_Consolidated")').iloc[-1]
                
                # 利益性チェック
                operating_income = int(latest_financial.get('OperatingProfit', 0))
                net_income = int(latest_financial.get('Profit', 0))
                
                if operating_income > 0:
                    financial_score += 8
                    reasons.append("営業利益が黒字")
                else:
                    reasons.append("営業利益が赤字（要注意）")
                
                if net_income > 0:
                    financial_score += 8
                    reasons.append("当期純利益が黒字")
                else:
                    reasons.append("当期純利益が赤字（要注意）")
                
                # 自己資本比率チェック
                equity_ratio = float(latest_financial.get('EquityToAssetRatio', 0)) * 100
                if equity_ratio > 30:
                    financial_score += 9
                    reasons.append(f"自己資本比率良好 ({equity_ratio:.1f}%)")
                elif equity_ratio > 10:
                    financial_score += 5
                    reasons.append(f"自己資本比率普通 ({equity_ratio:.1f}%)")
                else:
                    reasons.append(f"自己資本比率低い ({equity_ratio:.1f}%)")
            
            scores.append(("財務健全性", financial_score, 25))
            
            # 2. テクニカル分析スコア (0-25点)
            technical_score = 0
            if 'price_history' in self.analysis_results:
                hist = self.analysis_results['price_history']
                current_price = hist['Close'].iloc[-1]
                
                # 移動平均との比較
                sma20 = hist['Close'].rolling(window=20).mean().iloc[-1]
                sma50 = hist['Close'].rolling(window=50).mean().iloc[-1]
                
                if current_price > sma20:
                    technical_score += 8
                    reasons.append("株価が20日移動平均を上回る")
                
                if current_price > sma50:
                    technical_score += 8
                    reasons.append("株価が50日移動平均を上回る")
                
                # RSI分析
                delta = hist['Close'].diff()
                gain = (delta.where(delta > 0, 0)).rolling(window=14).mean()
                loss = (-delta.where(delta < 0, 0)).rolling(window=14).mean()
                rs = gain / loss
                rsi = 100 - (100 / (1 + rs))
                current_rsi = rsi.iloc[-1]
                
                if 30 <= current_rsi <= 70:
                    technical_score += 9
                    reasons.append(f"RSI適正範囲 ({current_rsi:.1f})")
                elif current_rsi < 30:
                    technical_score += 5
                    reasons.append(f"RSI売られすぎ水準 ({current_rsi:.1f}) - 反発期待")
                else:
                    reasons.append(f"RSI買われすぎ水準 ({current_rsi:.1f}) - 要注意")
            
            scores.append(("テクニカル", technical_score, 25))
            
            # 3. 流動性スコア (0-20点)
            liquidity_score = 0
            if 'price_history' in self.analysis_results:
                hist = self.analysis_results['price_history']
                avg_volume = hist['Volume'].mean()
                
                if avg_volume > 1000000:
                    liquidity_score += 20
                    reasons.append("出来高十分（流動性良好）")
                elif avg_volume > 100000:
                    liquidity_score += 15
                    reasons.append("出来高普通（流動性普通）")
                elif avg_volume > 10000:
                    liquidity_score += 10
                    reasons.append("出来高やや少ない")
                else:
                    liquidity_score += 5
                    reasons.append("出来高少ない（流動性リスク）")
            
            scores.append(("流動性", liquidity_score, 20))
            
            # 4. 市場評価スコア (0-15点)
            market_score = 0
            if 'recommendations' in self.analysis_results:
                rec_data = self.analysis_results['recommendations']
                
                # 買い推奨率に基づくスコア
                if rec_data['buy_ratio'] > 70:
                    market_score += 15
                    reasons.append("アナリスト：非常に強い買い推奨")
                elif rec_data['buy_ratio'] > 50:
                    market_score += 12
                    reasons.append("アナリスト：買い推奨")
                elif rec_data['buy_ratio'] > 30:
                    market_score += 8
                    reasons.append("アナリスト：やや買い推奨")
                
                # 売り推奨率のペナルティ
                if rec_data['sell_ratio'] > 70:
                    market_score -= 10
                    reasons.append("アナリスト：非常に強い売り推奨（注意）")
                elif rec_data['sell_ratio'] > 50:
                    market_score -= 7
                    reasons.append("アナリスト：売り推奨（注意）")

            
            if 'institutional_holders' in self.analysis_results:
                holders_count = len(self.analysis_results['institutional_holders'])
                if holders_count > 10:
                    market_score += 5
                    reasons.append(f"機関投資家保有多数 ({holders_count}社)")
                elif holders_count > 5:
                    market_score += 3
                    reasons.append(f"機関投資家保有あり ({holders_count}社)")
            
            scores.append(("市場評価", market_score, 15))
            
            # 5. 業界・セクタースコア (0-15点)
            sector_score = 10  # デフォルトスコア
            if 'yf_info' in self.analysis_results:
                sector = self.analysis_results['yf_info'].get('sector', '')
                if 'Technology' in sector:
                    sector_score += 5
                    reasons.append("テクノロジーセクター（成長期待）")
            
            scores.append(("業界評価", sector_score, 15))
            
            # 総合スコア計算
            total_score = sum([score for _, score, _ in scores])
            max_score = sum([max_val for _, _, max_val in scores])
            percentage_score = (total_score / max_score) * 100
            
            print(f"総合スコア: {total_score}/{max_score} ({percentage_score:.1f}%)")
            print("\n【スコア内訳】")
            for category, score, max_val in scores:
                print(f"  {category}: {score}/{max_val} ({(score/max_val)*100:.1f}%)")
            
            print(f"\n【評価理由】")
            for reason in reasons:
                print(f"  • {reason}")
            
            # 投資推奨度判定
            if percentage_score >= 80:
                recommendation = "強い買い推奨"
                risk_level = "低リスク"
            elif percentage_score >= 65:
                recommendation = "買い推奨"
                risk_level = "中リスク"
            elif percentage_score >= 50:
                recommendation = "条件付き買い"
                risk_level = "中リスク"
            elif percentage_score >= 35:
                recommendation = "様子見"
                risk_level = "高リスク"
            else:
                recommendation = "買い非推奨"
                risk_level = "高リスク"
            
            print(f"\n【総合判定】")
            print(f"投資推奨度: {recommendation}")
            print(f"リスクレベル: {risk_level}")
            
            self.analysis_results['investment_score'] = {
                'total_score': total_score,
                'max_score': max_score,
                'percentage': percentage_score,
                'recommendation': recommendation,
                'risk_level': risk_level,
                'detailed_scores': scores,
                'reasons': reasons
            }
            
        except Exception as e:
            print(f"投資スコア計算エラー: {e}")
            print("エラー情報\n" + traceback.format_exc())
    
    def run_full_analysis(self):
        """完全分析実行"""
        print(f"=== 東証株式{self.stock_code} 総合分析開始 ===\n")
        
        # 各分析を順次実行
        self.get_basic_info()
        self.get_financial_data()
        self.get_stock_price_analysis()
        self.get_market_sentiment()
        self.calculate_investment_score()
        
        print("\n" + "="*60)
        print("分析完了！")
        print("="*60)
        
        return self.analysis_results

# メイン実行部分
if __name__ == "__main__":
    # 株式コード3917で分析実行
    analyzer = StockAnalyzer("3236")
    
    try:
        results = analyzer.run_full_analysis()
        
        # 結果の保存
        print(f"\n分析結果を 'stock_analysis_3917.txt' に保存中...")
        
        with open("stock_analysis_3917.txt", "w", encoding="utf-8") as f:
            f.write("=== 東証株式3917 分析結果 ===\n")
            f.write(f"分析日時: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n\n")
            
            if 'investment_score' in results:
                score_data = results['investment_score']
                f.write(f"総合投資判定: {score_data['recommendation']}\n")
                f.write(f"投資スコア: {score_data['percentage']:.1f}%\n")
                f.write(f"リスクレベル: {score_data['risk_level']}\n\n")
                
                f.write("スコア詳細:\n")
                for category, score, max_val in score_data['detailed_scores']:
                    f.write(f"  {category}: {score}/{max_val}\n")
                
                f.write("\n評価理由:\n")
                for reason in score_data['reasons']:
                    f.write(f"  • {reason}\n")
        
        print("分析完了！ファイルに結果を保存しました。")
        
    except Exception as e:
        print(f"分析実行エラー: {e}")
        print("環境変数の設定やAPI接続を確認してください。")